# Run the forward model

emissions through to temperature

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import scipy.linalg
import scipy.stats
from scipy.interpolate import interp1d
import json
from tqdm import tqdm

from fair21.constants.gases import molwt, burden_per_emission, lifetime, gas_list
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon
)
from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha,
    step_concentration_1box,
    step_concentration_co2
)

from fair21.constants.general import (
    NBOX,
    EARTH_RADIUS,
    SECONDS_PER_YEAR,
    DOUBLING_TIME_1PCT
)
from fair21.defaults.gases import pre_industrial_concentration
from fair21.forcing.ghg import meinshausen

In [ ]:
"""
FaIR model interface
"""

class EnergyBalanceModel:
    """Energy balance model that converts forcing to temperature.

    The energy balance model is converted to an impulse-response formulation
    (hence the IR part of FaIR) to allow efficient evaluation. The benefits of
    this are increased as once derived, the "layers" of the energy balance
    model do not communicate with each other.

    Attributes
    ----------

    References
    ----------

    .. [1] Leach et al. (2021): https://doi.org/10.5194/gmd-14-3007-2021

    .. [2] Cummins et al. (2020): https://doi.org/10.1175/JCLI-D-19-0589.1

    .. [3] Tsutsui (2017): https://doi.org/10.1007/s10584-016-1832-9

    .. [4] Geoffroy et al. (2013): https://doi.org/10.1175/JCLI-D-12-00196.1
    """
    
    def __init__(self, **kwargs):
        """Initialise the EnergyBalanceModel.

        Parameters
        ----------
        **kwargs : dict, optional
            Parameters to run the energy balance model with.

            ocean_heat_capacity : `np.ndarray`
                Ocean heat capacity of each layer (top first), W m-2 yr K-1
            ocean_heat_transfer : `np.ndarray`
                Heat exchange coefficient between ocean layers (top first). The
                first element of this array is akin to the climate feedback
                parameter, with the convention that stabilising feedbacks are
                positive (opposite to most climate sensitivity literature).
                W m-2 K-1
            deep_ocean_efficacy : float
                efficacy of deepest ocean layer. See e.g. [1]_.
            forcing_4co2 : float
                effective radiative forcing from a quadrupling of atmospheric
                CO2 concentrations above pre-industrial.
            stochastic_run : bool
                Activate the stochastic variability component from [2]_.
            sigma_eta : float
                Standard deviation of stochastic forcing component from [2]_.
            sigma_xi : float
                Standard deviation of stochastic disturbance applied to surface
                layer. See [2]_.
            gamma_autocorrelation : float
                Stochastic forcing continuous-time autocorrelation parameter. 
                See [2]_.
            seed : int or None
                Random seed to use for stochastic variability.
                
        References
        ----------
        .. [1] Geoffroy, O., Saint-Martin, D., Bellon, G., Voldoire, A., Olivié,
            D. J. L., & Tytéca, S. (2013). Transient Climate Response in a Two-
            Layer Energy-Balance Model. Part II: Representation of the Efficacy
            of Deep-Ocean Heat Uptake and Validation for CMIP5 AOGCMs, Journal 
            of Climate, 26(6), 1859-1876

        .. [2] Cummins, D. P., Stephenson, D. B., & Stott, P. A. (2020). Optimal
            Estimation of Stochastic Energy Balance Model Parameters, Journal of
            Climate, 33(18), 7909-7926.
        """
        self.ocean_heat_capacity = kwargs.get('ocean_heat_capacity', np.array([5, 20, 100]))
        self.ocean_heat_transfer = kwargs.get('ocean_heat_transfer', np.array([1, 2, 1]))
        self.deep_ocean_efficacy = kwargs.get('deep_ocean_efficacy', 1)
        self.forcing_4co2 = kwargs.get('forcing_4co2', 8)
        self.stochastic_run = kwargs.get('stochastic_run', False)
        self.sigma_eta = kwargs.get('sigma_eta', 0.5)
        self.sigma_xi = kwargs.get('sigma_xi', 0.5)
        self.gamma_autocorrelation = kwargs.get('gamma_autocorrelation', 2)
        self.seed = kwargs.get('seed', None)

    def _eb_matrix(self):
        """Define the matrix of differential equations.

        Returns
        -------
        eb_matrix_eigenvalues : `np.ndarray`
            1D array of eigenvalues of the energy balance matrix.
        eb_matrix_eigenvectors : `np.ndarray`
            2D array of eigenvectors (an array of 1D eigenvectors) of the
            energy balance matrix.
        """
        eb_matrix = np.array(
            [
                [
                    -(self.ocean_heat_transfer[0]+self.ocean_heat_transfer[1])/self.ocean_heat_capacity[0],
                    self.ocean_heat_transfer[1]/self.ocean_heat_capacity[0], 
                    0
                ],
                [
                    self.ocean_heat_transfer[1]/self.ocean_heat_capacity[1],
                    -(self.ocean_heat_transfer[1]+self.deep_ocean_efficacy*self.ocean_heat_transfer[2])/self.ocean_heat_capacity[1],
                    self.deep_ocean_efficacy*self.ocean_heat_transfer[2]/self.ocean_heat_capacity[1]
                ],
                [
                    0, 
                    self.ocean_heat_transfer[2]/self.ocean_heat_capacity[2],
                    -self.ocean_heat_transfer[2]/self.ocean_heat_capacity[2]
                ]
            ]
        )
        return(eb_matrix)
    
        
    def impulse_response(self):
        """Converts the energy balance to impulse response."""
        eb_matrix = self._eb_matrix()

        # calculate the eigenvectors and eigenvalues, these are the timescales of responses
        eb_matrix_eigenvalues, eb_matrix_eigenvectors = scipy.linalg.eig(eb_matrix)
        self.timescales = -1/(np.real(eb_matrix_eigenvalues))
        self.response_coefficients = self.timescales * (eb_matrix_eigenvectors[0,:] * scipy.linalg.inv(eb_matrix_eigenvectors)[:,0]) / self.ocean_heat_capacity[0]

        
    def emergent_parameters(self):
        """Calculates emergent parameters from the energy balance parameters."""
        # requires impulse response step
        if not hasattr(self, 'timescales'):
            self.impulse_response()
        self.ecs = self.forcing_4co2 * forcing_2co2_4co2_ratio * np.sum(self.response_coefficients)
        self.tcr = self.forcing_4co2 * forcing_2co2_4co2_ratio * np.sum(
            self.response_coefficients*(
                1 - self.timescales/DOUBLING_TIME_1PCT * (
                    1 - np.exp(-DOUBLING_TIME_1PCT/self.timescales)
                )
            )
        )
    
    def add_forcing(self, forcing, time):
        self.forcing = forcing
        self.time = time
    
    def run(self):
        # eventually, we'll need a separate case for when forcing is not known in advance, or where there's
        # temperature feedbacks on the forcing
        # this is a key difference between a simple EBM and FaIR!

        # internal variables
        nmatrix = NBOX + self.stochastic_run
        n_timesteps = len(self.time)
        
        # Define the forcing vector
        forcing_vector = np.array([1/self.ocean_heat_capacity[0], 0, 0])
    
        # Prepend eb_matrix with stochastic terms if this is a stochastic run: Cummins et al. (2020) eqs. 13 and 14
        eb_matrix = self._eb_matrix()
        if self.stochastic_run:
            eb_matrix = np.insert(eb_matrix, 0, np.zeros(NBOX), axis=0)
            prepend_col = np.zeros(nmatrix)
            prepend_col[0] = -self.gamma_autocorrelation
            prepend_col[1] = 1/self.ocean_heat_capacity[0]
            eb_matrix = np.insert(eb_matrix, 0, prepend_col, axis=1)
            forcing_vector = np.zeros(nmatrix)
            forcing_vector[0] = self.gamma_autocorrelation
    
        # Calculate the matrix exponential
        eb_matrix_d = scipy.linalg.expm(eb_matrix)
    
        # Solve for temperature
        forcing_vector_d = scipy.linalg.solve(eb_matrix, (eb_matrix_d - np.identity(nmatrix)) @ forcing_vector)
    
        # define stochastic matrix
        stochastic_d = np.zeros((n_timesteps, nmatrix))
    
        # stochastic stuff
        if self.stochastic_run:
            q_mat = np.zeros((nmatrix, nmatrix))
            q_mat[0,0] = self.sigma_eta**2
            q_mat[1,1] = (self.sigma_xi/self.ocean_heat_capacity[0])**2
            ## use Van Loan (1978) to compute the matrix exponential
            h_mat = np.zeros((nmatrix*2, nmatrix*2))
            h_mat[:nmatrix,:nmatrix] = -eb_matrix
            h_mat[:nmatrix,nmatrix:] = q_mat
            h_mat[nmatrix:,nmatrix:] = eb_matrix.T
            g_mat = scipy.linalg.expm(h_mat)
            q_mat_d = g_mat[nmatrix:,nmatrix:].T @ g_mat[:nmatrix,nmatrix:]
            q_mat_d = q_mat_d.astype(np.float64)
            stochastic_d = scipy.stats.multivariate_normal.rvs(
                size=n_timesteps, mean=np.zeros(nmatrix), cov=q_mat_d, random_state=self.seed
            )
        solution = np.zeros((n_timesteps, nmatrix))
        for i in range(1, n_timesteps):
            solution[i, :] = eb_matrix_d @ solution[i-1, :] + forcing_vector_d * self.forcing[i-1] + stochastic_d[i-1, :]

        if self.stochastic_run:
            self.temperature = solution[:, 1:]
            self.stochastic_forcing = solution[:, 0]
        else:
            self.temperature = solution
        self.toa_imbalance = self.forcing - self.ocean_heat_transfer[0]*self.temperature[:,0] + (1 - self.deep_ocean_efficacy) * self.ocean_heat_transfer[2] * (self.temperature[:,1] - self.temperature[:,2])
        self.ocean_heat_content_change = np.cumsum(self.toa_imbalance * np.concatenate(([0], np.diff(self.time)))) * EARTH_RADIUS**2 * 4 * np.pi * SECONDS_PER_YEAR

In [ ]:
# grab some emissions
emissions = {}
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for gas in gas_list:
    gas_rcmip_name = gas.replace("-", "")
    emissions[gas] = df.loc[
        (df['Scenario']=='ssp245') & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
    ].interpolate(axis=1).values.squeeze()

# CO2 and N2O units need to behave
emissions["CO2"] = emissions["CO2"] / 1000
emissions["N2O"] = emissions["N2O"] / 1000

In [ ]:
class Emissions:
    
    def __init__(self, emissions):
        self.emissions = emissions
        
    def from_csv(self):
        pass
        
    def to_csv(self):
        pass

In [ ]:
scen1 = Emissions()

In [ ]:
scen1.emissions